# Importation des packages nécessaires

In [1]:
from bs4 import BeautifulSoup as bs
# Bibliothèque BeautifulSoup permettant d'analyser et d'extraire des données d'une page HTML de manière structurée.
# Importée sous l'alias `bs` pour simplifier son utilisation.

import os
# Module intégré permettant de manipuler le système d'exploitation, comme la gestion des fichiers et des dossiers.

from openpyxl import Workbook
# Bibliothèque pour créer et manipuler des fichiers Excel (XLSX).
# `Workbook` est utilisé pour créer un nouveau classeur Excel.

from selenium import webdriver
# Bibliothèque Selenium permettant l'automatisation du navigateur.
# `webdriver` est utilisé pour initialiser et contrôler le navigateur.

from selenium.webdriver.common.by import By
# Permet de localiser les éléments sur une page Web à l'aide de différents sélecteurs (ID, XPATH, CSS_SELECTOR, etc.).

from selenium.webdriver.support.ui import WebDriverWait
# Offre la possibilité de gérer les temps d'attente explicites pour des éléments spécifiques sur la page avant d'interagir avec eux.

from selenium.webdriver.support import expected_conditions as EC
# Définit des conditions d'attente (comme la visibilité ou la cliquabilité d'un élément) pour automatiser la navigation.

from selenium.common.exceptions import TimeoutException
# Exception levée lorsque le temps d'attente d'un élément est dépassé, utile pour gérer les erreurs de chargement d'éléments.

from openpyxl import load_workbook
# Bibliothèque pour manipuler des fichiers Excel existants.
# `load_workbook` est utilisé pour charger un classeur Excel existant et modifier son contenu.

import time
# Module intégré pour la gestion des opérations temporelles, offrant des fonctions comme `time.sleep()` pour ajouter des pauses.


# Création du dossier où enregistrer le fichier Excel

Ce code définit et crée un dossier nommé `Fichiers_Excel` pour stocker les fichiers Excel générés lors de l'exécution du programme.

Processus :

- Attribue à ``excel_folder`` le chemin du dossier où les fichiers Excel seront enregistrés.
- Utilise ``os.makedirs()`` pour créer le dossier s'il n'existe pas déjà, grâce à l'option ``exist_ok=True``, qui évite toute erreur si le dossier est déjà présent.

In [2]:
# Dossier où enregistrer le fichier Excel
excel_folder = 'Fichiers_Excel'
# Définit le nom du dossier où les fichiers Excel seront enregistrés.

os.makedirs(excel_folder, exist_ok=True)
# Crée le dossier spécifié s'il n'existe pas déjà.
# L'option `exist_ok=True` empêche une erreur si le dossier existe déjà.


# Fonction pour nettoyer le texte

Remplace les caractères spéciaux et nettoie le texte

In [3]:
def clean_text(text):
    return text.replace('\xa0', ' ').strip()

# Fonction pour l'exportation des données vers Excel

La fonction ``export_to_excel`` enregistre les données fournies dans un fichier Excel, ajoutant les nouvelles lignes à la suite des données existantes. Elle gère la création d’un fichier si celui-ci n’existe pas et intègre les en-têtes nécessaires.

Paramètres :

- ``data`` : Liste de listes contenant les informations à enregistrer, où chaque sous-liste représente une ligne.

Processus :

Définit le nom du fichier Excel et les en-têtes de colonne.

Vérifie si le fichier Excel existe déjà :

- Si oui, ouvre le fichier et sélectionne la feuille de données.
- Sinon, crée un nouveau fichier, définit le titre de la feuille active et ajoute les en-têtes de colonne.
- Pour chaque ligne de données, nettoie les éléments et les ajoute dans le fichier.
- Sauvegarde les modifications dans le fichier Excel et affiche un message de succès.
- Gère les erreurs en affichant un message d’erreur si l’exportation échoue.

In [4]:
def export_to_excel(data):
    """Exporte les données dans un fichier Excel en ajoutant de nouvelles données à la suite des anciennes."""

    try:
        excel_filename = os.path.join(excel_folder, 'data_Mooc_batiment_durable.xlsx')
        # Définit le chemin complet du fichier Excel où les données seront enregistrées.

        headers = [
            'Thème', 'Référence', 'Titre', 'Durée', "Temps d'effort", 'Rythme', 'Langue', 'Inscription', 
            'Cours', 'Texte', 'Plan de Formation', 'Sessions de Cours Archivées', 'Lien', 
            'Propriétaire vidéo', 'Titre vidéo', 'Nombre de commentaires', 'Nombre de vues', 'Nombre de likes', 
            'Description vidéo', 'Date publication video', 'Lien de la vidéo'
        ]
        # Définition des en-têtes pour chaque colonne du fichier Excel.

        # Si le fichier existe, charger le classeur existant
        if os.path.exists(excel_filename):
            wb = load_workbook(excel_filename)
            ws_general = wb['Informations générales']
            # Charge le fichier Excel existant et sélectionne la feuille intitulée "Informations générales".
        else:
            # Créer un nouveau classeur Excel et ajouter les en-têtes
            wb = Workbook()
            ws_general = wb.active
            ws_general.title = 'Informations générales'
            ws_general.append(headers)
            # Crée un nouveau fichier Excel, nomme la feuille active "Informations générales" et ajoute les en-têtes.

        # Écrire les données dans la feuille Excel à la suite des données existantes
        for line in data:
            cleaned_line = [clean_text(str(item)) for item in line]
            ws_general.append(cleaned_line)
            # Pour chaque ligne de données, convertit les éléments en chaînes, les nettoie, et les ajoute à la suite des données existantes.

        # Sauvegarder les modifications
        wb.save(excel_filename)
        print(f"Données exportées avec succès vers : {excel_filename}")
        # Sauvegarde le fichier Excel après l'ajout de nouvelles données et affiche un message de confirmation.

    except Exception as e:
        print(f"Une erreur est survenue lors de l'exportation vers Excel : {str(e)}")
        # Affiche un message d'erreur en cas d'exception lors de l'exportation des données.


# Fonction pour rejeter les coockies

La fonction ``click_refuse_cookies`` interagit avec une page web pour refuser les cookies lorsque le bouton de refus est présent et cliquable. Elle utilise des attentes explicites pour s'assurer de la visibilité et de la cliquabilité du bouton avant d'exécuter le clic.

Paramètres :

- ``driver`` : Instance du WebDriver Selenium utilisée pour interagir avec la page et exécuter le clic.

Processus :

- Attend jusqu’à 20 secondes que le bouton de refus des cookies soit présent dans la page.
- Scrolle la page pour garantir la visibilité du bouton.
- Attend jusqu'à 10 secondes supplémentaires que le bouton soit cliquable.
- Clique sur le bouton pour refuser les cookies.
- En cas de succès, affiche un message de confirmation ; sinon, affiche un message d'erreur en cas d'exception.

In [5]:
def click_refuse_cookies(driver):
    # Cette fonction clique sur le bouton "Refuser" des cookies lorsqu'il est présent sur une page web.

    try:
        refuse_button = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, "//button[@aria-label=\"Refuser l'utilisation de cookies et d'autres données aux fins décrites\"]"))
        )
        # Attendre jusqu'à 20 secondes pour que le bouton "Refuser" des cookies soit présent dans la page.

        driver.execute_script("arguments[0].scrollIntoView(true);", refuse_button)
        # Faire défiler la page pour s'assurer que le bouton est visible à l'écran.

        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//button[@aria-label=\"Refuser l'utilisation de cookies et d'autres données aux fins décrites\"]"))
        )
        # Attendre jusqu'à 10 secondes pour que le bouton soit cliquable.

        refuse_button.click()
        # Cliquer sur le bouton pour refuser les cookies.

        print("Bouton 'Tout refuser' des cookies cliqué")
        # Message de confirmation indiquant que l'action de refus des cookies a été effectuée.

    except Exception as e:
        print("Erreur lors du refus des cookies")
        # Affiche un message d'erreur en cas d'exception lors du clic sur le bouton de refus des cookies.


# Fonction pour récupérer le nombre de commentaires d'une vidéo

La fonction ``get_nombre_commentaire`` utilise Selenium pour charger une vidéo YouTube, scroller jusqu’au bas de la page pour s’assurer que les commentaires sont chargés, et ensuite extraire le nombre de commentaires si ceux-ci sont activés.

Paramètres :

- ``driver`` : Instance du WebDriver Selenium pour interagir avec la page de la vidéo.
- ``video_url`` : URL de la vidéo YouTube pour laquelle on veut récupérer le nombre de commentaires.

Processus :

- Charge la page de la vidéo et fait défiler vers le bas pour assurer le chargement des commentaires.
- Attend que le corps de la page soit visible pour garantir le chargement de tous les éléments.
- Vérifie si les commentaires sont désactivés :
- Si oui, retourne "N/A" et affiche un message indiquant leur désactivation.
- Si non, tente d'extraire le nombre de commentaires affiché.
- Gère les erreurs en affichant un message et en retournant "N/A" si une exception survient.

In [6]:
def get_nombre_commentaire(driver, video_url):
    # Charge la page d'une vidéo YouTube et extrait le nombre de commentaires si ceux-ci sont activés.

    try:
        # Charger la page de la vidéo
        driver.get(video_url)
        # Charge la page de la vidéo YouTube en utilisant l'URL spécifiée.

        # Scroller vers le bas de la page pour charger les commentaires
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        # Fait défiler la page vers le bas pour s'assurer que les commentaires sont chargés.

        # Attendre quelques secondes pour s'assurer que la page a bien défilé et que les commentaires sont chargés
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )
        # Attendre que le corps de la page soit chargé pour garantir la disponibilité de tous les éléments.

        # Essayer d'extraire le message indiquant que les commentaires sont désactivés
        try:
            commentaire_desactive_element = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.XPATH, "//yt-formatted-string[contains(text(), 'Les commentaires sont désactivés')]"))
            )
            if commentaire_desactive_element:
                nombre_commentaires = "N/A"
                print("Les commentaires sont désactivés sur cette vidéo.")
                # Si un message indiquant la désactivation des commentaires est trouvé, retourne "N/A".

        except TimeoutException:
            # Si les commentaires ne sont pas désactivés, extraire le nombre de commentaires
            try:
                nombre_commentaires = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'yt-formatted-string.count-text.style-scope.ytd-comments-header-renderer'))
                ).text
                # Récupère le texte indiquant le nombre de commentaires affiché sur la page.

            except:
                nombre_commentaires = "N/A"
                print("Erreur lors de l'extraction du nombre de commentaires")
                # En cas d'erreur, retourne "N/A".

    except:
        nombre_commentaires = "N/A"
        print("Erreur lors du chargement de la vidéo ")
        # En cas d'erreur lors du chargement de la vidéo, retourne "N/A".

    return nombre_commentaires
    # Retourne le nombre de commentaires ou "N/A" si les commentaires sont désactivés ou si une erreur survient.


# Fonction pour récuperer le titre d'une vidéo

La fonction ``get_titre_video`` utilise Selenium pour charger la page d'une vidéo YouTube et en extraire le titre. Elle attend que l'élément contenant le titre soit visible avant de récupérer son texte.

Paramètres :

- ``driver`` : Instance du WebDriver Selenium pour accéder à la page de la vidéo et interagir avec les éléments.
- ``video_url`` : URL de la vidéo YouTube dont on veut extraire le titre.

Retour :

Renvoie le titre de la vidéo sous forme de texte. Si le titre n'est pas disponible ou si une erreur survient, retourne "N/A".

Processus :

- Charge la page de la vidéo et attend que l'élément contenant le titre soit présent.
- Localise et extrait le texte du titre en s'assurant qu'il n'est pas vide.
- Gère les erreurs en affichant un message et en retournant "N/A" si l'élément du titre est introuvable ou si une exception se produit.

In [7]:
def get_titre_video(driver, video_url):
    # Charge la page d'une vidéo YouTube et extrait le titre de la vidéo si celui-ci est disponible.

    try:
        driver.get(video_url)
        # Charge la page de la vidéo YouTube à partir de l'URL spécifiée.

        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'div.style-scope.ytd-watch-metadata yt-formatted-string.style-scope.ytd-watch-metadata'))
        )
        # Attend jusqu'à 20 secondes pour que l'élément contenant le titre de la vidéo soit présent sur la page.

        titre_video_element = driver.find_element(By.CSS_SELECTOR, 'div.style-scope.ytd-watch-metadata yt-formatted-string.style-scope.ytd-watch-metadata')
        # Localise l'élément contenant le titre de la vidéo en utilisant le sélecteur CSS approprié.

        return titre_video_element.text.strip() if titre_video_element else 'N/A'
        # Retourne le texte du titre de la vidéo sans espaces de début ou de fin.
        # Si l'élément du titre n'est pas trouvé, retourne "N/A".

    except Exception as e:
        print(f"Erreur lors de la récupération du titre de la vidéo : {str(e)}")
        return 'N/A'
        # Affiche un message d'erreur et retourne "N/A" en cas d'exception lors de l'extraction du titre.


# Fonction pour récupérer le propriétaire de la vidéo

La fonction ``get_video_owner`` utilise Selenium pour charger la page d'une vidéo YouTube et en extraire le nom du propriétaire de la chaîne. Elle attend que l'élément contenant le nom soit visible avant de le récupérer.

Paramètres :

- ``driver`` : Instance du WebDriver Selenium pour accéder à la page de la vidéo et interagir avec les éléments.
- ``video_url`` : URL de la vidéo YouTube dont on souhaite extraire le nom du propriétaire.

Retour :

Renvoie le nom du propriétaire de la chaîne sous forme de texte. Si le nom n'est pas disponible ou qu'une erreur survient, retourne "N/A".

Processus :

- Charge la page de la vidéo et attend que l'élément contenant le nom du propriétaire soit présent.
- Localise et extrait le texte du nom du propriétaire.
- Gère les erreurs en affichant un message et en retournant "N/A" si l'élément du propriétaire est introuvable ou si une exception se produit.

In [8]:
def get_video_owner(driver, video_url):
    # Charge la page d'une vidéo YouTube et extrait le nom du propriétaire de la chaîne si celui-ci est disponible.

    try:
        driver.get(video_url)
        # Charge la page de la vidéo YouTube à partir de l'URL spécifiée.

        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'div.style-scope ytd-channel-name a.yt-simple-endpoint.style-scope.yt-formatted-string'))
        )
        # Attend jusqu'à 20 secondes pour que l'élément contenant le nom du propriétaire soit présent sur la page.

        owner_element = driver.find_element(By.CSS_SELECTOR, 'div.style-scope ytd-channel-name a.yt-simple-endpoint.style-scope.yt-formatted-string')
        # Localise l'élément contenant le nom du propriétaire de la chaîne YouTube.

        return owner_element.text.strip() if owner_element else 'N/A'
        # Retourne le nom du propriétaire sans espaces de début ou de fin.
        # Si l'élément du propriétaire n'est pas trouvé, retourne "N/A".

    except Exception as e:
        print(f"Erreur lors de la récupération du propriétaire de la vidéo : {str(e)}")
        return 'N/A'
        # Affiche un message d'erreur et retourne "N/A" en cas d'exception lors de l'extraction du nom du propriétaire.


# Fonction pour récupérer le nombre de vue d'une vidéo

La fonction ``get_nombre_vues`` utilise Selenium pour charger la page d'une vidéo YouTube et en extraire le nombre de vues. Elle attend que l'élément contenant le nombre de vues soit visible avant de le récupérer.

Paramètres :

- ``driver`` : Instance du WebDriver Selenium pour accéder à la page de la vidéo et interagir avec les éléments.
- ``video_url`` : URL de la vidéo YouTube dont on veut extraire le nombre de vues.

Retour :

Renvoie le nombre de vues de la vidéo sous forme de texte. Si le nombre de vues n'est pas disponible ou si une erreur survient, retourne "N/A".

Processus :

- Charge la page de la vidéo et attend que l'élément contenant le nombre de vues soit présent.
- Localise et extrait le texte du nombre de vues.
- Gère les erreurs en affichant un message et en retournant "N/A" si l'élément du nombre de vues est introuvable ou si une exception se produit.

In [9]:
def get_nombre_vues(driver, video_url):
    # Charge la page d'une vidéo YouTube et extrait le nombre de vues si celui-ci est disponible.

    try:
        driver.get(video_url)
        # Charge la page de la vidéo YouTube à partir de l'URL spécifiée.

        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'yt-formatted-string.style-scope.ytd-watch-info-text span.style-scope.yt-formatted-string.bold'))
        )
        # Attend jusqu'à 20 secondes pour que l'élément contenant le nombre de vues soit présent sur la page.

        vues_element = driver.find_element(By.CSS_SELECTOR, 'yt-formatted-string.style-scope.ytd-watch-info-text span.style-scope.yt-formatted-string.bold')
        # Localise l'élément contenant le nombre de vues de la vidéo.

        return vues_element.text.strip() if vues_element else 'N/A'
        # Retourne le texte du nombre de vues sans espaces de début ou de fin.
        # Si l'élément du nombre de vues n'est pas trouvé, retourne "N/A".

    except Exception as e:
        print(f"Erreur lors de la récupération du nombre de vues : {str(e)}")
        return 'N/A'
        # Affiche un message d'erreur et retourne "N/A" en cas d'exception lors de l'extraction du nombre de vues.


# Fonction pour récupérer le nombre de likes d'une vidéo

La fonction ``get_nombre_likes`` utilise Selenium pour charger la page d'une vidéo YouTube et en extraire le nombre de likes. Elle attend que l'élément contenant les likes soit visible avant de le récupérer et vérifie si le nombre de likes est effectivement affiché.

Paramètres :

- ``drive``r : Instance du WebDriver Selenium pour accéder à la page de la vidéo et interagir avec les éléments.
- ``video_url`` : URL de la vidéo YouTube dont on souhaite extraire le nombre de likes.

Retour :

Renvoie le nombre de likes sous forme de texte. Si le nombre de likes n'est pas visible ou qu'une erreur survient, retourne "N/A".

Processus :

- Charge la page de la vidéo et attend que l'élément contenant le nombre de likes soit présent.
- Localise et extrait le texte du nombre de likes.
- Si le texte contient l’indication "J'aime", retourne "N/A".
- Gère les erreurs en affichant un message et en retournant "N/A" si l'élément du nombre de likes est introuvable ou si une exception se produit.

In [10]:
def get_nombre_likes(driver, video_url):
    # Charge la page d'une vidéo YouTube et extrait le nombre de likes si celui-ci est disponible.

    try:
        # Chargement de la page vidéo
        driver.get(video_url)
        # Charge la page de la vidéo YouTube à partir de l'URL spécifiée.

        # Attente que l'élément des likes soit présent
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/div[2]/ytd-watch-metadata/div/div[2]/div[2]/div/div/ytd-menu-renderer/div[1]/segmented-like-dislike-button-view-model/yt-smartimation/div/div/like-button-view-model/toggle-button-view-model/button-view-model/button/div[2]'))
        )
        # Attend jusqu'à 20 secondes que l'élément contenant le nombre de likes soit visible sur la page.

        # Récupération de l'élément contenant les likes
        likes_element = driver.find_element(By.XPATH, '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/div[2]/ytd-watch-metadata/div/div[2]/div[2]/div/div/ytd-menu-renderer/div[1]/segmented-like-dislike-button-view-model/yt-smartimation/div/div/like-button-view-model/toggle-button-view-model/button-view-model/button/div[2]')
        # Localise l'élément contenant le nombre de likes en utilisant le sélecteur XPath.

        # Vérification si l'élément contient le texte "J'aime"
        likes_text = likes_element.text.strip()
        if "J'aime" in likes_text:
            return 'N/A'
        # Si l'élément contient le texte "J'aime", retourne 'N/A' car cela indique qu'il n'y a pas de nombre de likes visible.

        # Retour du nombre de likes si disponible, sinon 'N/A'
        return likes_text if likes_text else 'N/A'
        # Retourne le texte du nombre de likes. Si le texte est vide, retourne "N/A".

    except Exception as e:
        print(f"Erreur lors de la récupération du nombre de likes : {str(e)}")
        return 'N/A'
        # Affiche un message d'erreur et retourne "N/A" en cas d'exception lors de l'extraction du nombre de likes.


# Fonction pour récupérer la description d'une vidéo

La fonction ``get_description_video`` utilise Selenium pour charger la page d'une vidéo YouTube et extraire la description complète. Elle clique d'abord sur le bouton "Afficher plus" pour s'assurer que la description entière est visible, puis la récupère.

Paramètres :

- ``driver`` : Instance du WebDriver Selenium pour accéder à la page de la vidéo et interagir avec les éléments.
- ``video_url`` : URL de la vidéo YouTube dont on souhaite extraire la description.

Retour :

Renvoie la description complète de la vidéo sous forme de texte. Si la description n'est pas disponible ou si une erreur survient, retourne "NA".

Processus :

- Charge la page de la vidéo et attend que le bouton "Afficher plus" soit cliquable, puis clique dessus pour étendre la description.
- Attends brièvement pour assurer le chargement complet de la description.
- Localise et extrait le texte de la description.
- Gère les erreurs en affichant un message et en retournant "N/A" si l'élément de description est introuvable ou si une exception se produit.

In [11]:
def get_description_video(driver, video_url):
    # Charge la page d'une vidéo YouTube et extrait la description complète de la vidéo si celle-ci est disponible.

    try:
        driver.get(video_url)
        # Charge la page de la vidéo YouTube à partir de l'URL spécifiée.

        # Cliquer sur le bouton "Afficher plus" pour afficher toute la description
        show_more_button_xpath = '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/div[2]/ytd-watch-metadata/div/div[4]/div[1]/div/ytd-text-inline-expander/tp-yt-paper-button[1]'
        show_more_button = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, show_more_button_xpath))
        )
        driver.execute_script("arguments[0].click();", show_more_button)
        # Attend jusqu'à 20 secondes que le bouton "Afficher plus" soit cliquable, puis clique pour étendre la description.

        time.sleep(2)  # Pause pour laisser le temps à la description complète de se charger
        # Attente de 2 secondes pour garantir le chargement complet de la description étendue.

        # Récupérer l'élément contenant la description complète
        description_video_element = driver.find_element(By.CSS_SELECTOR, 'span.yt-core-attributed-string.yt-core-attributed-string--white-space-pre-wrap')
        description = description_video_element.text.strip() if description_video_element else 'N/A'
        # Localise l'élément contenant la description complète et extrait son texte, ou retourne "N/A" si absent.

        # Si la description est vide, retourner 'N/A'
        return description if description else 'N/A'
        # Retourne le texte de la description si elle est non vide, sinon retourne "N/A".

    except Exception as e:
        print(f"Erreur lors de la récupération de la description : {str(e)}")
        return 'N/A'
        # Affiche un message d'erreur et retourne "N/A" en cas d'exception lors de l'extraction de la description.


# Fonction pour récupérer la date de publication d'une vidéo

La fonction ``get_date_publication`` utilise Selenium pour charger la page d'une vidéo YouTube et en extraire la date de publication. Elle attend que l'élément contenant la date soit visible avant de récupérer son texte.

Paramètres :

- `driver` : Instance du WebDriver Selenium pour accéder à la page de la vidéo et interagir avec les éléments.
- ``video_url`` : URL de la vidéo YouTube dont on souhaite extraire la date de publication.

Retour :

Renvoie la date de publication sous forme de texte. Si la date n'est pas disponible ou si une erreur survient, retourne "N/A".

Processus :

- Charge la page de la vidéo et attend que l'élément contenant la date de publication soit présent.
- Localise et extrait le texte de la date de publication.
- Gère les erreurs en affichant un message et en retournant "N/A" si l'élément de date est introuvable ou si une exception se produit.

In [12]:
def get_date_publication(driver, video_url):
    # Charge la page d'une vidéo YouTube et extrait la date de publication si celle-ci est disponible.

    try:
        driver.get(video_url)
        # Charge la page de la vidéo YouTube à partir de l'URL spécifiée.

        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/div[2]/ytd-watch-metadata/div/div[4]/div[1]/div/ytd-watch-info-text/div/yt-formatted-string/span[3]'))
        )
        # Attend jusqu'à 20 secondes que l'élément contenant la date de publication soit visible sur la page.

        date_element = driver.find_element(By.XPATH, '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-watch-flexy/div[5]/div[1]/div/div[2]/ytd-watch-metadata/div/div[4]/div[1]/div/ytd-watch-info-text/div/yt-formatted-string/span[3]')
        # Localise l'élément contenant la date de publication en utilisant le sélecteur XPath.

        return date_element.text.strip() if date_element else 'N/A'
        # Retourne le texte de la date de publication sans espaces de début ou de fin.
        # Si l'élément de date est introuvable, retourne "N/A".

    except Exception as e:
        print(f"Erreur lors de la récupération de la date de publication : {str(e)}")
        return 'N/A'
        # Affiche un message d'erreur et retourne "N/A" en cas d'exception lors de l'extraction de la date de publication.


# Récupération de tous les liens

La fonction ``get_pagination_urls`` utilise Selenium pour récupérer les liens de pagination des pages de résultats, en sélectionnant chaque lien de la barre de pagination et en les ajoutant à une liste.

Paramètres :

- ``driver`` : Instance du WebDriver Selenium pour accéder à la page de résultats et interagir avec les éléments de pagination.

Retour :

Renvoie une liste de liens (pagination_links) pour chaque page de résultats. Si aucune pagination n’est disponible ou en cas d'erreur, retourne une liste vide.

Processus :

- Sélectionne tous les éléments de lien de pagination et extrait l’attribut href de chaque lien.
- Ajoute chaque lien unique dans la liste pagination_links pour éviter les doublons.
- Gère les erreurs en affichant un message et continue avec une liste vide si une exception se produit.

In [13]:
def get_pagination_urls(driver):
    
    pagination_links = []  # Liste pour stocker les liens de pagination.

    try:
        # Trouver tous les éléments <li> de pagination contenant des liens
        pagination_items = driver.find_elements(By.CSS_SELECTOR, "li.pagination__item a")
        # Sélectionne tous les éléments <a> à l'intérieur des éléments <li> avec la classe "pagination__item".

        for item in pagination_items:
            href = item.get_attribute("href")
            if href and href not in pagination_links:
                pagination_links.append(href)
                # Ajoute le lien à la liste s'il n'est pas déjà présent pour éviter les doublons.

    except Exception as e:
        print(f"Erreur lors de la récupération des liens de pagination : {str(e)}")
        # Affiche un message d'erreur en cas d'exception lors de la récupération des liens.

    return pagination_links
    # Retourne la liste des liens de pagination.


# Fonction pour extraire les informations d'une page web

Chargement de la page : 

La fonction commence par charger l'URL de la page web et attend que les éléments clés soient présents avant de continuer.

Extraction des informations principales :

-  ``themes``, ``reference``, ``titre``, `duree`, ``temps_d_effort``, ``rythme``, ``langue`` etc sont extraits via des sélecteurs CSS.

Extraction des informations supplémentaires :

- inscription et cours sont extraits à partir des éléments `<dl>`, ``<dt>``, et `<dd>`.

Extraction du texte, plan de formation et sessions archivées :

- Ces éléments sont extraits via des sélecteurs CSS spécifiques aux sections de texte, du plan de formation et des sessions archivées.

Gestion de la vidéo :

- Si une vidéo est trouvée dans un iframe, les informations sur cette vidéo sont extraites (propriétaire, titre, vues, likes, etc.).

Retour : 

- La fonction retourne une liste contenant toutes les informations extraites ou une liste de ['N/A'] en cas d'erreur.

In [14]:
def get_info(link, driver):

    
    try:
        # Charger la page avec Selenium
        print(f"Chargement de la page : {link}")
        driver.get(link)
        # Charge la page web à partir de l'URL donnée.

        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.course-detail__aside")))
        # Attend jusqu'à 10 secondes que l'élément clé de la page (ici un élément <div> spécifique) soit présent.

        soup = bs(driver.page_source, 'html.parser')
        # Utilise BeautifulSoup pour analyser le contenu HTML de la page chargée par Selenium.

        def extract_text(selector, default='N/A'):
            """Fonction interne pour extraire du texte à partir d'un sélecteur CSS avec une valeur par défaut."""
            element = soup.select_one(selector)
            return element.get_text(strip=True) if element else default
            # Retourne le texte de l'élément sélectionné si trouvé, sinon retourne la valeur par défaut ('N/A').

        # Extraction des différentes informations
        themes = ', '.join([span.get_text() for span in soup.select('div.category-badge-list__container a.category-badge span.category-badge__title')]) or 'N/A'
        # Extrait tous les thèmes à partir des éléments <span> avec une classe spécifique, les joint avec des virgules ou retourne 'N/A' si aucun n'est trouvé.

        reference = extract_text('div.subheader__code')
        # Extrait le texte de l'élément contenant le code de référence du cours.

        titre = extract_text('h1.subheader__title')
        # Extrait le titre du cours à partir de l'élément <h1>.

        characteristics_items = soup.select('div.characteristics ul.characteristics__container li.characteristics__item span.characteristics__term')
        # Sélectionne tous les éléments <span> dans la section des caractéristiques du cours.

        # Extraction de certaines caractéristiques spécifiques
        duree = next((item.get_text(strip=True) for item in characteristics_items if "Durée" in item.get_text(strip=True)), 'N/A')
        # Cherche la caractéristique "Durée" parmi les éléments sélectionnés.

        temps_d_effort = next((item.get_text(strip=True) for item in characteristics_items if "Effort" in item.get_text(strip=True)), 'N/A')
        # Cherche la caractéristique "Effort" (temps d'effort nécessaire) parmi les éléments.

        rythme = next((item.get_text(strip=True) for item in characteristics_items if "Rythme" in item.get_text(strip=True)), 'N/A')
        # Cherche la caractéristique "Rythme" parmi les éléments.

        langue = next((item.get_text(strip=True) for item in characteristics_items if "Langue" in item.get_text(strip=True)), 'N/A')
        # Cherche la caractéristique "Langue" parmi les éléments.


        # Extraire les informations supplémentaires
        inscription = 'N/A'
        cours = 'N/A'
        # Initialise les variables pour l'inscription et les informations sur le cours à 'N/A' par défaut.

        dl_elements = soup.select('div.course-detail__run-descriptions.course-detail__run-descriptions--course_and_search dl')
        # Sélectionne tous les éléments <dl> dans la section des descriptions de cours.

        if dl_elements:
            for dl in dl_elements:
                dts = dl.find_all('dt')
                dds = dl.find_all('dd')
                # Pour chaque <dl>, extrait tous les éléments <dt> (titres) et <dd> (descriptions).

                for i in range(len(dts)):
                    dt_text = dts[i].get_text(strip=True)
                    # Récupère le texte de chaque élément <dt>.

                    if 'Inscription' in dt_text and i + 1 < len(dds):
                        inscription = dds[i].get_text(strip=True)
                        # Si l'élément <dt> contient le mot "Inscription", récupère le texte associé de <dd>.

                    elif 'Cours' in dt_text and i + 1 < len(dds):
                        cours = dds[i].get_text(strip=True)
                        # Si l'élément <dt> contient le mot "Cours", récupère le texte associé de <dd>.

        # Extraction du texte descriptif principal
        texte = ' '.join([element.get_text(strip=True) for element in soup.select('div.course-detail__block.course-detail__primary-group p')]) or 'N/A'
        # Sélectionne tous les paragraphes <p> dans la section principale du cours, extrait et joint leur texte, ou retourne 'N/A' s'il n'y a pas de texte disponible.

        # Extraction du plan de formation
        plan_de_formation = ' '.join([element.get_text(strip=True) for element in soup.select('section.course-detail__row.course-detail__plan div.nested-item__content')]) or 'N/A'
        # Sélectionne tous les éléments <div> dans la section du plan de formation, extrait et joint leur texte, ou retourne 'N/A' s'il n'y a pas de plan.

        # Extraction des sessions de cours archivées
        sessions_de_cours_archivées = ' '.join([element.get_text(strip=True) for element in soup.select('div.course-detail__row.course-detail__runs.course-detail__runs--archived ul li')]) or 'N/A'
        # Sélectionne tous les éléments <li> dans la section des sessions archivées, extrait et joint leur texte, ou retourne 'N/A' s'il n'y a pas de sessions archivées.


        # Informations sur la vidéo
        Propriétaire_video = 'N/A'
        Titre_vidéo = 'N/A'
        Nb_commentaire_video = 'N/A'
        Nb_vues_video = 'N/A'
        Nb_likes_video = 'N/A'
        Description_video = 'N/A'
        Date_publication_video = 'N/A'
        Lien_video = 'N/A'

        # Trouver le lien de l'iframe
        iframe = soup.find('iframe', {'title': 'Vidéo'})
        if iframe:
            iframe_src = iframe.get('src')
            print(f"IFrame src trouvé: {iframe_src}")
            if iframe_src:
                if not iframe_src.startswith('http'):
                    iframe_src = 'https:' + iframe_src  # Assurez-vous que l'URL commence par 'https:'
                driver.get(iframe_src)
                time.sleep(3)
                video_soup = bs(driver.page_source, 'html.parser')
                
                # Trouver le lien de la vidéo après avoir cliqué sur l'iframe
                video_link = video_soup.find('a', class_='ytp-impression-link')
                if video_link:
                    Lien_video = video_link.get('href')
                    print(f"Video URL trouvé: {Lien_video}")  # Affiche les href récupérés
                    if Lien_video:
                        driver.get(Lien_video)
                        time.sleep(3)
                        # Vérifier si la vidéo est disponible
                        if "cette vidéo n'est plus disponible" not in driver.page_source.lower():
                            Propriétaire_video = get_video_owner(driver, Lien_video)
                            Titre_vidéo = get_titre_video(driver, Lien_video)
                            Nb_commentaire_video = get_nombre_commentaire(driver, Lien_video)
                            Nb_vues_video = get_nombre_vues(driver, Lien_video)
                            Nb_likes_video = get_nombre_likes (driver, Lien_video)
                            Description_video = get_description_video(driver, Lien_video)
                            Date_publication_video = get_date_publication(driver, Lien_video)
                        else:
                            print("Cette vidéo n'est plus disponible.")
                            Lien_video = 'N/A'

        line = [
            themes,
            reference,
            titre,
            duree,
            temps_d_effort,
            rythme,
            langue,
            inscription,
            cours,
            texte,
            plan_de_formation,
            sessions_de_cours_archivées,
            link,
            Propriétaire_video,
            Titre_vidéo,
            Nb_commentaire_video,
            Nb_vues_video,
            Nb_likes_video,
            Description_video,
            Date_publication_video,
            Lien_video
        ]

        print(f"Données extraites : {line}")  # Vérification des données extraites
        return line

    except Exception as e:
        print(f"Une erreur est survenue lors de l'extraction des informations : {str(e)}")
        return ['N/A'] * 20  # Retourne une liste de N/A

# Fonction pour 

Thèmes à rechercher :

- Une liste de thèmes (``themes = ["Métiers de l'encadrement", "bas carbone"``]) est définie, qui peut être modifiée pour inclure d'autres thèmes.

Initialisation du dossier Excel :

- Le dossier ``Fichiers_Excel`` est créé pour stocker le fichier Excel. Si un fichier avec le même nom existe déjà, il est supprimé.

Initialisation du navigateur :

Selenium est utilisé pour automatiser le processus de recherche et extraction des données sur les formations en utilisant un WebDriver (Chrome dans cet exemple).

Recherche et extraction des liens :

- Pour chaque thème, la fonction accède à une URL de base, entre le thème dans la barre de recherche et clique sur le bouton de recherche pour afficher les résultats.
- Les liens des formations affichées sur la première page sont récupérés. Ensuite, la pagination est gérée pour extraire les liens des formations sur les autres pages de résultats.

Pagination :

- La fonction `get_pagination_urls(driver)` est appelée pour récupérer les URLs des pages suivantes dans les résultats, et chaque page est visitée pour extraire les nouveaux liens de formations.

Extraction des informations :

- Une fois les liens de formations récupérés, la fonction ``get_info(href, driver)`` est appelée pour extraire les informations détaillées de chaque formation (par exemple, titre, durée, description, etc.).

Exportation des données :

- Toutes les informations extraites sont ensuite exportées dans un fichier Excel via la fonction ``export_to_excel(all_data)``.

Gestion du navigateur :

- À la fin du processus, le navigateur Selenium est fermé pour libérer les ressources.

## Points importants :

- Pagination : La fonction gère la navigation entre plusieurs pages de résultats, ce qui permet d'extraire un grand nombre de formations.
- Robustesse : Les fonctions de Selenium attendent que les éléments clés soient chargés avant de continuer, ce qui garantit la stabilité du processus d'extraction.
- Exportation : Les informations extraites sont sauvegardées dans un fichier Excel pour une analyse ou un usage ultérieur.

In [ ]:
def extract_urls_and_info():
    

    # Liste des thèmes à rechercher
    themes = ["Métiers de l'encadrement", "bas carbone"]  # Vous pouvez ajouter d'autres thèmes ici

    # Chemin vers le dossier où enregistrer le fichier Excel
    dossier_excel = 'Fichiers_Excel'
    os.makedirs(dossier_excel, exist_ok=True)  # Créer le dossier s'il n'existe pas
    excel_path = os.path.join(dossier_excel, 'data_Mooc_batiment_durable.xlsx')

    # Supprimer l'ancien fichier Excel s'il existe avant de traiter le premier thème
    if os.path.exists(excel_path):
        os.remove(excel_path)

    # Initialisation du navigateur avec Selenium
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(options=options)

    try:
        for theme in themes:
            print(f"Traitement du thème : {theme}")
            base_url = "https://www.mooc-batiment-durable.fr/fr/formations/?limit=21&offset=0&query={}"
            driver.get(base_url.format(theme))

            # Attendre que l'élément de recherche soit chargé
            WebDriverWait(driver, 80).until(EC.presence_of_element_located((By.CSS_SELECTOR, "input.react-autosuggest__input")))

            search_input = driver.find_element(By.CSS_SELECTOR, "input.react-autosuggest__input")
            search_input.clear()
            time.sleep(10)  # Pause pour s'assurer que la page est bien chargée
            search_input.send_keys(theme)  # Entrer le thème dans la barre de recherche

            search_button = driver.find_element(By.CSS_SELECTOR, "svg.icon.icon--medium.search-input__btn__icon")
            search_button.click()  # Cliquer sur le bouton de recherche

            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.course-glimpse-list__content")))

            # Extraire les URLs des formations sur la première page
            hrefs = []
            course_glimpses = driver.find_elements(By.CSS_SELECTOR, "div.course-glimpse__media a")
            for glimpse in course_glimpses:
                hrefs.append(glimpse.get_attribute("href"))  # Ajouter les liens des formations à la liste

            print(f"Liens récupérés sur la première page : {len(hrefs)}")

            # Récupérer les liens de pagination via les éléments <li class="pagination__item">
            pagination_links = get_pagination_urls(driver)

            # Parcourir chaque lien de pagination et extraire les données des formations
            for page_link in pagination_links:
                driver.get(page_link)  # Aller à la page suivante
                time.sleep(3)  # Attendre que la page se charge
                course_glimpses = driver.find_elements(By.CSS_SELECTOR, "div.course-glimpse__media a")
                for glimpse in course_glimpses:
                    href = glimpse.get_attribute("href")
                    if href not in hrefs:  # Ajouter les nouveaux liens non encore récupérés
                        hrefs.append(href)

                print(f"Liens récupérés sur la page {page_link} : {len(hrefs)}")

            # Extraire les informations de chaque lien de formation
            all_data = []
            for href in hrefs:
                data = get_info(href, driver)  # Appel de la fonction pour extraire les informations
                all_data.append(data)

            # Exporter les données dans un fichier Excel
            export_to_excel(all_data)

            print(f"Nombre total de liens récupérés pour le thème '{theme}' : {len(hrefs)}")

    finally:
        driver.quit()  # Fermer le navigateur à la fin du processus

if __name__ == "__main__":
    extract_urls_and_info()


Traitement du thème : Métiers de l'encadrement
Liens récupérés sur la première page : 9
Chargement de la page : https://www.mooc-batiment-durable.fr/fr/formations/les-metiers-de-la-renovation-ont-besoin-de-vous/
Données extraites : ['Construire durable, Rénovation', 'Réf. AI_ENVIRONNEMENT_C1', 'Les métiers de la rénovation ont besoin de vous', 'N/A', 'N/A', 'N/A', 'N/A', 'Du 04 janv. 2023 au 01 févr. 2030', 'Du 14 févr. 2023 au 15 avr. 2030', "Le MOOC « Les métiers de la rénovation énergétique ont besoin de vous » est développé par AI Environnement, groupe FACEA pour le compte de l’ADEME, deux acteurs engagés pour la transition énergétique et environnementale des bâtiments.Il permet de découvrir les métiers liés à la rénovation énergétique des bâtiments et de défaire certains préjugés tenaces. Il vient mettre en valeur et faire découvrir concrètement onze métiers et à travers des interviews d’artisans et autres professionnels et des images de leur quotidien. En outre, les trois webinai